In [ ]:
%pip install requests click

In [ ]:
import requests
import click

api_key = "CG-YgaSDYyvsiDQsZvTB8KNjJcR"

headers = {
    "x-cg-demo-api-key": api_key
}

@click.group()
def cli():
    pass

def get_coin_price(coin_id, currency):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies={currency}"
    data = requests.get(url, headers=headers).json()
    return data[coin_id][currency]

@click.command()
@click.option("--coin_id", default="bitcoin")
@click.option("--currency", default="usd")
def show_coin_price(coin_id, currency):
    coin_price = get_coin_price(coin_id, currency)
    print(f"The price of {coin_id} is {coin_price:.2f} {currency.upper()}")

cli.add_command(show_coin_price)
# Instead of using sys.argv, directly call the function with arguments
cli.main(args=['show-coin-price', '--coin_id', 'ethereum', '--currency', 'usd'])


In [ ]:
import sqlite3

database = sqlite3.connect("portfolio.db") #Pass in the DB file name, if it does not exist, just like CLI it will be created.

cursor = database.cursor()

create_table_query = """
CREATE TABLE investments(
    coin_id TEXT,
    currency TEXT,
    sell INT,
    amount REAL,
    date TIMESTAMP
);
"""

cursor.execute(create_table_query)
# sell determines if we are buying or selling, however SQLite does not have a data type of boolean so we have to use INT, The sqlite3 python module is smart enough to convert Python bools into SQLite int whenever we are serialzing into the database.
#Same goes for TIMESTAMP. Python date datatypes are implicitly converted to the TIMESTAMPS SQLite Data type.

In [ ]:
import datetime

investments = ("bitcoin", "usd", True, 1.0, datetime.datetime.now()) #We use tuples usually to define DB entries as we don't want them to be mutable

cursor.execute(
    "INSERT INTO investments VALUES (?, ?, ?, ?, ?);", #using ? we can pass arguments to the SQL query. ? acts as a placeholder.
    investments
)

database.commit()

result = cursor.execute("SELECT * FROM investments;")

print(result.fetchone())

In [ ]:
import requests
import click
import sqlite3
import datetime

api_key = "CG-YgaSDYyvsiDQsZvTB8KNjJcR"

headers = {
    "x-cg-demo-api-key": api_key
}

# Establish database connection
database = sqlite3.connect("portfolio.db")
cursor = database.cursor()

# Create table if not exists
create_table_query = """
CREATE TABLE IF NOT EXISTS investments(
    coin_id TEXT,
    currency TEXT,
    amount REAL,
    sell INTEGER,
    date TIMESTAMP
);
"""
cursor.execute(create_table_query)
database.commit()

#CLick is the pythonic way to create a command line interface for your scripts or application. It's smart and easy to use and creates arguments automatically for the decorators defined.
# Under the hood it has a CLI API that has a commandline parser that talks with the interpreter than works completely different than optparse or argparse built in libraries. It gives you easy ways to create highly nested and complex commands with options.
# CLick also provides out of the box support for reading env variables form values and lazy loading and file handling.
# You transform methods into click commands using decorators.

#Create a command group, a group can have multiple commands assocated with it. The group name will the function name
@click.group()
def cli():
    pass

def get_coin_price(coin_id, currency):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies={currency}"
    data = requests.get(url, headers=headers).json()
    return data[coin_id][currency]

@cli.command() #Command decorator that aslo assigns this command to the group.
@click.option("--coin_id", default="bitcoin") #Adding an option
@click.option("--currency", default="usd")
def get_investment_value(coin_id, currency): #the option name gets passed onto the function parameter name. Then the function uses iut wthin the code.
    coin_price = get_coin_price(coin_id, currency)
    sql = """SELECT amount
    FROM investments
    WHERE coin_id=?
    AND currency=?
    AND sell=?;"""
    buy_result = cursor.execute(sql, (coin_id, currency, False)).fetchall()
    sell_result = cursor.execute(sql, (coin_id, currency, True)).fetchall()
    
    buy_amount = sum(row[0] for row in buy_result) #The default data structure returned is a list of tuples [(row1Data,...),(row2Data....)], as we are just querying the amounts cilumn the row data will be a tuple with single entry at row[0]
    sell_amount = sum(row[0] for row in sell_result)
    
    total = buy_amount - sell_amount
    
    print(f"You own a total of {total} {coin_id} worth {total * coin_price} {currency.upper()}")

@cli.command()
@click.option("--coin_id", default="bitcoin")
@click.option("--currency", default="usd")
def show_coin_price(coin_id, currency):
    coin_price = get_coin_price(coin_id, currency)
    print(f"The price of {coin_id} is {coin_price:.2f} {currency.upper()}")
    
@cli.command()
@click.option("--coin_id")
@click.option("--currency")
@click.option("--amount", type=float)
@click.option("--sell", is_flag=True)
def add_investments(coin_id, currency, amount, sell):
    try:
        sql = "INSERT INTO investments (coin_id, currency, amount, sell, date) VALUES (?, ?, ?, ?, ?);"
        values = (coin_id, currency, amount, int(sell), datetime.datetime.now())
        cursor.execute(sql, values)
        database.commit()
        print(f"Added {'sell' if sell else 'buy'} of {amount} {coin_id}")
    except Exception as e:
        print(f"An error occurred: {e}")
        database.rollback()

# Function to run commands in Jupyter
def run_command(command, *args):
    try:
        cli.main(args=[command] + list(args), standalone_mode=False)
    except SystemExit:
        pass

# Example usage in Jupyter
run_command('show-coin-price', '--coin_id', 'ethereum', '--currency', 'usd')
run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '1.0')
#selling some btc
run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '0.5', '--sell')

#Getting investment value
run_command('get-investment-value', '--coin_id', 'bitcoin', '--currency', 'inr')

The price of ethereum is 2495.60 USD
Added buy of 1.0 bitcoin
Added sell of 0.5 bitcoin
You own a total of 4.5 bitcoin worth 40061772.0 INR


C:\Users\adminuser\AppData\Local\Temp\2\ipykernel_2632\3726229988.py:76: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute(sql, values)


In [ ]:
#Up untill now we are defining strict tuples and using positional arguments to set the values in the database. This worked for a Poc project, but for complex projects it would be difficult
# to create an maintain and is very confusing We cannot use Generic collecctions like List or Tuples. Is there a way to use our own custom data structures that we create using Classes and objects?
#the answer is yes, using ROW FACTORIES. It also let's us refer to values using object names rather than position in some generic collection.

#The database has a row_factory attribute.
#sqlite3 includes a row row_factory attribute
# This transforms a tuple into a dictionary. With the DB columns as dictionary keys. Can also use namedtuples or collections.

#The pythonic design to wrap the row data is preferebly a data class. It is a quick way to create a class that intends to mainly depict the data structure of a Database row.

from dataclasses import dataclass

@dataclass
class Investment:
    coin_id: str
    currency: str
    amount: float
    sell: bool
    date: datetime.datetime
    
    def compute_value(self):
        return self.amount * get_coin_price(self.coin_id, self.currency)
    

#Create a row factory that maps a Database 'row' tuple items like row[0] (coin_id), row[1](currency), row[2](amount) etc. to the dataclass attributes and instantiate the dataclass object with a partcilar row's data.
#Row Factor function
def investment_row_factory(_, row): #Using a throwaway parameter as a placeholder for cursor in this example. The row factory usually takes the cursor and the row as parameter
    return Investment(
        coin_id=row[0],
        currency=row[1],
        amount=row[2],
        sell=bool(row[3]),
        date=datetime.datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f")
    )
    
database.row_factory = investment_row_factory #setting the row factory for the database session.

#A query like below
    
# sql = """SELECT *
#     FROM investments;"""

# Will return by default a list of rows, with each row as a tuple containing data. So a list of tuples. This is hard to parse and get data from.
#With a row factory defined on the database, the database will now by default run the factory logic when you query the database and return a python Investment object.



In [10]:
#Implementation with row factory
import requests
import click
import sqlite3
import datetime

api_key = "CG-YgaSDYyvsiDQsZvTB8KNjJcR"

headers = {
    "x-cg-demo-api-key": api_key
}

#Data class to define a data structure for the Investments table row.
@dataclass
class Investment:
    coin_id: str
    currency: str
    amount: float
    sell: bool
    date: datetime.datetime
    
    def compute_value(self):
        return self.amount * get_coin_price(self.coin_id, self.currency)
    
#Create a row factory that maps a Database 'row' tuple items like row[0] (coin_id), row[1](currency), row[2](amount) etc. to the dataclass attributes and instantiate the dataclass object with a partcilar row's data.
#Row Factor function
def investment_row_factory(_, row): #Using a throwaway parameter as a placeholder for cursor in this example. The row factory usually takes the cursor and the row as parameter
    return Investment(
        coin_id=row[0],
        currency=row[1],
        amount=row[2],
        sell=bool(row[3]),
        date=datetime.datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S.%f")
    )

# Establish database connection
database = sqlite3.connect("portfolio.db")
database.row_factory = investment_row_factory #setting the row factory for the database session. Make sure to set this before cursor otherwise it will still ouput tuples.
cursor = database.cursor()

# Create table if not exists
create_table_query = """
CREATE TABLE IF NOT EXISTS investments(
    coin_id TEXT,
    currency TEXT,
    amount REAL,
    sell INTEGER,
    date TIMESTAMP
);
"""
cursor.execute(create_table_query)
database.commit()

#CLick is the pythonic way to create a command line interface for your scripts or application. It's smart and easy to use and creates arguments automatically for the decorators defined.
# Under the hood it has a CLI API that has a commandline parser that talks with the interpreter than works completely different than optparse or argparse built in libraries. It gives you easy ways to create highly nested and complex commands with options.
# CLick also provides out of the box support for reading env variables form values and lazy loading and file handling.
# You transform methods into click commands using decorators.

#Create a command group, a group can have multiple commands assocated with it. The group name will the function name
@click.group()
def cli():
    pass

def get_coin_price(coin_id, currency):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies={currency}"
    data = requests.get(url, headers=headers).json()
    return data[coin_id][currency]

@cli.command() #Command decorator that aslo assigns this command to the group.
@click.option("--coin_id", default="bitcoin") #Adding an option
@click.option("--currency", default="usd")
def get_investment_value(coin_id, currency): #the option name gets passed onto the function parameter name. Then the function uses iut wthin the code.
    coin_price = get_coin_price(coin_id, currency) #Now we can select all the columns and not amounts as we have a data factory
    sql = """SELECT *
    FROM investments
    WHERE coin_id=?
    AND currency=?
    AND sell=?;"""
    buy_result = cursor.execute(sql, (coin_id, currency, False)).fetchall()
    sell_result = cursor.execute(sql, (coin_id, currency, True)).fetchall()
    
    buy_amount = sum(row.amount for row in buy_result) #With data factory implemented we now get row Investmeent objects that has 'amount' attribute. So now we can use dot notation.
    sell_amount = sum(row.amount for row in sell_result)
    
    total = buy_amount - sell_amount
    
    print(f"You own a total of {total} {coin_id} worth {total * coin_price} {currency.upper()}")

@cli.command()
@click.option("--coin_id", default="bitcoin")
@click.option("--currency", default="usd")
def show_coin_price(coin_id, currency):
    coin_price = get_coin_price(coin_id, currency)
    print(f"The price of {coin_id} is {coin_price:.2f} {currency.upper()}")
    
@cli.command()
@click.option("--coin_id")
@click.option("--currency")
@click.option("--amount", type=float)
@click.option("--sell", is_flag=True)
def add_investments(coin_id, currency, amount, sell):
    try:
        sql = "INSERT INTO investments (coin_id, currency, amount, sell, date) VALUES (?, ?, ?, ?, ?);"
        values = (coin_id, currency, amount, int(sell), datetime.datetime.now())
        cursor.execute(sql, values)
        database.commit()
        print(f"Added {'sell' if sell else 'buy'} of {amount} {coin_id}")
    except Exception as e:
        print(f"An error occurred: {e}")
        database.rollback()

# Function to run commands in Jupyter
def run_command(command, *args):
    try:
        cli.main(args=[command] + list(args), standalone_mode=False)
    except SystemExit:
        pass

# Example usage in Jupyter
run_command('show-coin-price', '--coin_id', 'ethereum', '--currency', 'usd')
run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '1.0')
#selling some btc
run_command('add-investments', '--coin_id', 'bitcoin', '--currency', 'inr', '--amount', '0.5', '--sell')

#Getting investment value
run_command('get-investment-value', '--coin_id', 'bitcoin', '--currency', 'inr')

The price of ethereum is 2500.48 USD
Added buy of 1.0 bitcoin
Added sell of 0.5 bitcoin
You own a total of 5.0 bitcoin worth 44540070.0 INR


C:\Users\adminuser\AppData\Local\Temp\2\ipykernel_2632\3209646770.py:105: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute(sql, values)
